In [1]:
from utils.data_handling import *
from utils.sensor_optimisation_gpy_sparse import *
from parameters import *
import GPy

import sklearn.decomposition
import scipy
import matplotlib.pyplot as plt
np.random.seed(101)
%matplotlib inline

from tqdm import tqdm

In [2]:
parameters['i_end'] = 988
parameters['field_name'] = "Tracer"
parameters

{'i_start': 0,
 'i_end': 988,
 'crop': None,
 'field_name': 'Tracer',
 'cov_method': 'sample'}

In [3]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

100%|██████████| 1/1 [00:00<00:00,  5.13it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


In [ ]:
def human_subset(loc_df, h):
    
    
    human_selection = []
    return human_selection

In [10]:
main_df = loc_df.copy()

In [26]:
main_df.loc[:,'Xr'] = loc_df.loc[:,['X']].round(0)
main_df.loc[:,'Yr'] = loc_df.loc[:,['Y']].round(0)
min_surface = main_df.groupby(['Xr','Yr']).min().loc[:,'Z']

(43632,)

In [265]:
def set_to_onehot(A,n):
    """Function that maps a list of points to a one hot encoding of selected points"""
    A_list = np.zeros((n,1))
    A_list[A] = 1
    return A_list

In [271]:
A_list = set_to_onehot(A,data_df.shape[0])
save_vtu(ref_vtu[0],'WorkingSelection10-4.vtu',A_list)

==> Saved to : ../data/temp_data/LSBU_res_2019:07:08-17:41:27_WorkingSelection10-4.vtu
